# Regge-Wheeler and Zerilli equations for the Scharzschild BH

**Reference:** 

Michele Maggiore, Gravitational Waves, Volume 2, Oxford University Press, 2018

In [1]:
# %display latex
import time
comput_time0 = time.perf_counter()
stage = 0
total_time = []

## Basic setup

In [2]:
var('l m M')
cl = var('cl',latex_name=r'l(l+1)')
sep_const = {cl:l*(l+1)}
RW = Manifold(4,'RW',structure='Lorentzian')
Ch.<t,r,θ,ϕ> = RW.chart()
RW.set_simplify_function(lambda q: q)
g = RW.metric('g')
f = function('f')(r)
# B = function('B')(r)
f = 1-2*M/r
# α,ll = var('α ℓ')
# f = 1 - 2*(M + α*ll/2)/r + α*exp(-r/M)
B = 1/f
g[:] = matrix.diagonal([-f,B,r^2,(r*sin(θ))^2])
nabla = g.connection()
η = RW.metric('η')
η[:] = matrix.diagonal([-1,1,r^2,(r*sin(θ))^2])
show(η.display())
show(g.display())

η = -dt⊗dt + dr⊗dr + r^2 dθ⊗dθ + r^2*sin(θ)^2 dϕ⊗dϕ

g = (2*M/r - 1) dt⊗dt - 1/(2*M/r - 1) dr⊗dr + r^2 dθ⊗dθ + r^2*sin(θ)^2 dϕ⊗dϕ

In [3]:
from sage.manifolds.utilities import ExpressionNice as EN
def disp(ex):
    """
    Display symbolic expressions using the sagemanifolds textbook
    displaying style.
    """
    return show(EN(ex))

In [4]:
def count_time(t0,t1):
    return time.strftime("%H:%M:%S",time.gmtime(t1 - t0))

def stage_time():
    """
    Calculate and show the total time of the session. Add +1 to the 
    stage session counter at each call.
    """
    global stage
    total_time.append(count_time(comput_time0,time.perf_counter()))
    print(f"Total time elapsed until the end of stage {stage}: {total_time[stage]}")
    stage += 1

In [5]:
def to_table(dictionary, horizontal=False):
    """
    Display a dictionary as a table, with keys on the first column
    and values on the second.
    """
    keys, values = dictionary.keys(), dictionary.values()
    if horizontal:
        return table([list(keys), list(values)], header_row=True)
    else:
        return table(list(zip(keys,values)),header_column=True, align='left')

In [6]:
def mycollect(e1, e2):
    """
    Collects e2 coefficients in e1, even when e2 is not a factor of
    some of the terms of e1
    """
    g = maxima_calculus.gensym()._sage_()
    while g in e1.variables(): g = maxima_calculus.gensym()._sage_()
    return sum([v[0]*g^v[1]
                for v in e1.subs(e2==g).coefficients(g)]).subs(g==e2)

### Spherical harmonics

In [7]:
Y = function('Y', latex_name=r'Y_l^m')(θ,ϕ)
X = function('X', latex_name=r'X_l^m')(θ,ϕ)
W = function('W', latex_name=r'W_l^m')(θ,ϕ)
sph = spherical_harmonic(l,m,θ,ϕ)
from_XW = {X:2*diff(Y,θ,ϕ) - 2*cot(θ)*diff(Y,ϕ), 
         W:diff(Y,θ,2) - cot(θ)*diff(Y,θ) - (1/sin(θ))^2*diff(Y,ϕ,2)
          }
to_sph = {Y:sph, 
          X:2*diff(sph,θ,ϕ) - 2*cot(θ)*diff(sph,ϕ), 
          W:diff(sph,θ,2) - cot(θ)*diff(sph,θ) - (1/sin(θ))^2*diff(sph,ϕ,2)
         }
to_table(from_XW)

"\(X_l^m\left(θ, ϕ\right)\)","\(-2 \, \cot\left(θ\right) \frac{\partial}{\partial ϕ}Y_l^m\left(θ, ϕ\right) + 2 \, \frac{\partial^{2}}{\partial θ\partial ϕ}Y_l^m\left(θ, ϕ\right)\)"
"\(W_l^m\left(θ, ϕ\right)\)","\(-\cot\left(θ\right) \frac{\partial}{\partial θ}Y_l^m\left(θ, ϕ\right) - \frac{\frac{\partial^{2}}{(\partial ϕ)^{2}}Y_l^m\left(θ, ϕ\right)}{\sin\left(θ\right)^{2}} + \frac{\partial^{2}}{(\partial θ)^{2}}Y_l^m\left(θ, ϕ\right)\)"


In [8]:
for q in from_XW:
    print(latex((from_XW[q])))

-2 \, \cot\left(θ\right) \frac{\partial}{\partial ϕ}Y_l^m\left(θ, ϕ\right) + 2 \, \frac{\partial^{2}}{\partial θ\partial ϕ}Y_l^m\left(θ, ϕ\right)
-\cot\left(θ\right) \frac{\partial}{\partial θ}Y_l^m\left(θ, ϕ\right) - \frac{\frac{\partial^{2}}{(\partial ϕ)^{2}}Y_l^m\left(θ, ϕ\right)}{\sin\left(θ\right)^{2}} + \frac{\partial^{2}}{(\partial θ)^{2}}Y_l^m\left(θ, ϕ\right)


In [9]:
to_XW = solve(X == from_XW[X],diff(Y,θ,ϕ),solution_dict=True)[0]
to_XW.update(solve(W == from_XW[W],diff(Y,θ,2),solution_dict=True)[0])
to_table(to_XW)

"\(\frac{\partial^{2}}{\partial θ\partial ϕ}Y_l^m\left(θ, ϕ\right)\)","\(\cot\left(θ\right) \frac{\partial}{\partial ϕ}Y_l^m\left(θ, ϕ\right) + \frac{1}{2} \, X_l^m\left(θ, ϕ\right)\)"
"\(\frac{\partial^{2}}{(\partial θ)^{2}}Y_l^m\left(θ, ϕ\right)\)","\(\frac{\cot\left(θ\right) \sin\left(θ\right)^{2} \frac{\partial}{\partial θ}Y_l^m\left(θ, ϕ\right) + W_l^m\left(θ, ϕ\right) \sin\left(θ\right)^{2} + \frac{\partial^{2}}{(\partial ϕ)^{2}}Y_l^m\left(θ, ϕ\right)}{\sin\left(θ\right)^{2}}\)"


In [10]:
to_table(to_sph)

"\(Y_l^m\left(θ, ϕ\right)\)","\(Y_{l}^{m}\left(θ, ϕ\right)\)"
"\(X_l^m\left(θ, ϕ\right)\)","\(2 i \, m^{2} \cot\left(θ\right) Y_{l}^{m}\left(θ, ϕ\right) + 2 \, \sqrt{{\left(l + m + 1\right)} {\left(l - m\right)}} {\left(i \, m + i\right)} e^{\left(-i \, ϕ\right)} Y_{l}^{m + 1}\left(θ, ϕ\right) - 2 i \, m \cot\left(θ\right) Y_{l}^{m}\left(θ, ϕ\right) - 2 i \, \sqrt{{\left(l + m + 1\right)} {\left(l - m\right)}} e^{\left(-i \, ϕ\right)} Y_{l}^{m + 1}\left(θ, ϕ\right)\)"
"\(W_l^m\left(θ, ϕ\right)\)","\({\left(m \cot\left(θ\right) Y_{l}^{m}\left(θ, ϕ\right) + \sqrt{{\left(l + m + 1\right)} {\left(l - m\right)}} e^{\left(-i \, ϕ\right)} Y_{l}^{m + 1}\left(θ, ϕ\right)\right)} m \cot\left(θ\right) - {\left(\cot\left(θ\right)^{2} + 1\right)} m Y_{l}^{m}\left(θ, ϕ\right) + {\left({\left(m + 1\right)} \cot\left(θ\right) Y_{l}^{m + 1}\left(θ, ϕ\right) + \sqrt{{\left(l + m + 2\right)} {\left(l - m - 1\right)}} e^{\left(-i \, ϕ\right)} Y_{l}^{m + 2}\left(θ, ϕ\right)\right)} \sqrt{{\left(l + m + 1\right)} {\left(l - m\right)}} e^{\left(-i \, ϕ\right)} - {\left(m \cot\left(θ\right) Y_{l}^{m}\left(θ, ϕ\right) + \sqrt{{\left(l + m + 1\right)} {\left(l - m\right)}} e^{\left(-i \, ϕ\right)} Y_{l}^{m + 1}\left(θ, ϕ\right)\right)} \cot\left(θ\right) + \frac{m^{2} Y_{l}^{m}\left(θ, ϕ\right)}{\sin\left(θ\right)^{2}}\)"


### Laplace equation

In [11]:
Lap = 1/(Y*sin(θ))*diff(sin(θ)*diff(Y,θ),θ) + -m^2/(Y*sin(θ)^2)*Y==-cl
lap_eq = Lap.solve(diff(Y,θ,2),solution_dict=True)[0]
to_table(lap_eq)

"\(\frac{\partial^{2}}{(\partial θ)^{2}}Y_l^m\left(θ, ϕ\right)\)","\(-\frac{\cos\left(θ\right) \sin\left(θ\right) \frac{\partial}{\partial θ}Y_l^m\left(θ, ϕ\right) + {\left({l(l+1)} \sin\left(θ\right)^{2} - m^{2}\right)} Y_l^m\left(θ, ϕ\right)}{\sin\left(θ\right)^{2}}\)"


## Tensor harmonics

In [12]:
T = {'L0':matrix.diagonal([0,1,0,0])*Y,
     'T0':matrix.diagonal([0,0,1,sin(θ)^2])*Y,
     'E1':matrix([[0,0,0,0],[0,0,diff(Y,θ),diff(Y,ϕ)],[0,diff(Y,θ),0,0],[0,diff(Y,ϕ),0,0]]),
     'B1':matrix([[0,0,0,0],[0,0,1/sin(θ)*diff(Y,ϕ),-sin(θ)*diff(Y,θ)],[0,1/sin(θ)*diff(Y,ϕ),0,0],[0,-sin(θ)*diff(Y,θ),0,0]]),
     'E2':matrix([[0,0,0,0],[0,0,0,0],[0,0,W,X],[0,0,X,-sin(θ)^2*W]]),
     'B2':matrix([[0,0,0,0],[0,0,0,0],[0,0,-1/sin(θ)*X,sin(θ)*W],[0,0,sin(θ)*W,sin(θ)*X]]),
     'tt':matrix.diagonal([1,0,0,0])*Y,
     'Rt':matrix([[0,1,0,0],[1,0,0,0],[0,0,0,0],[0,0,0,0]])*Y,
     'Et':matrix([[0,0,diff(Y,θ),diff(Y,ϕ)],[0,0,0,0],[diff(Y,θ),0,0,0],[diff(Y,ϕ),0,0,0]]),
     'Bt':matrix([[0,0,1/sin(θ)*diff(Y,ϕ),-sin(θ)*diff(Y,θ)],[0,0,0,0],[1/sin(θ)*diff(Y,ϕ),0,0,0],[-sin(θ)*diff(Y,θ),0,0,0]])
    }
vector_index = ['Et','Bt','E1','B1']
tensor_index = ['E2','B2']
scalar_index = ['L0','T0','tt','Rt']
# to_table(T)

In [13]:
for ti in scalar_index: # nonzero for l=0
    print(ti,all([T[ti].apply_map(lambda q: q.substitute_function(to_sph))(l=0,m=0).simplify_trig() != 0]))

L0 True
T0 True
tt True
Rt True


In [14]:
for ti in vector_index: # vanishes for l<1
    print(ti,all([T[ti].apply_map(lambda q: q.substitute_function(to_sph))(l=0,m=0).simplify_trig() == 0]))

Et True
Bt True
E1 True
B1 True


In [15]:
for ti in tensor_index: # vanishes for l<2
    print(ti,all([T[ti].apply_map(lambda q: q.substitute_function(to_sph))(l=1,m=k).simplify_trig() == 0 for k in [-1,0,1]]))

E2 True
B2 True


In [16]:
TH = {q:RW.tensor_field(0,2,T[q], name=q,latex_name='(T^{'+q+'})') for q in T}
print(TH['Bt'])
show(TH['Bt'].display_comp())

Tensor field Bt of type (0,2) on the 4-dimensional Lorentzian manifold RW


Bt_tθ = d(Y)/dϕ/sin(θ) 
Bt_tϕ = -sin(θ)*d(Y)/dθ 
Bt_θt = d(Y)/dϕ/sin(θ) 
Bt_ϕt = -sin(θ)*d(Y)/dθ

In [17]:
# ((TH['Bt'].up(η)*TH['Bt'])['^ab_ab'].expr().substitute_function(to_sph)*sin(θ)).integral(θ,algorithm='fricas').integral(ϕ,algorithm='fricas')

In [18]:
# for ti in scalar_index:
#     for tj in scalar_index:
#         integral()

In [19]:
hlm = {k:function(r'h'+k,latex_name=r'\textit{h}_{lm}^{'+k+r'}')(t,r) for k in T}
to_table({q:(hlm[q]*TH[q])[:] for q in T})

L0,"\(\left(\begin{array}{rrrr} 0 & 0 & 0 & 0 \\ 0 & Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{L0}\left(t, r\right) & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \end{array}\right)\)"
T0,"\(\left(\begin{array}{rrrr} 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{T0}\left(t, r\right) & 0 \\ 0 & 0 & 0 & Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{T0}\left(t, r\right) \sin\left(θ\right)^{2} \end{array}\right)\)"
E1,"\(\left(\begin{array}{rrrr} 0 & 0 & 0 & 0 \\ 0 & 0 & \textit{h}_{lm}^{E1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & \textit{h}_{lm}^{E1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} \\ 0 & \textit{h}_{lm}^{E1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & 0 & 0 \\ 0 & \textit{h}_{lm}^{E1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} & 0 & 0 \end{array}\right)\)"
B1,"\(\left(\begin{array}{rrrr} 0 & 0 & 0 & 0 \\ 0 & 0 & \frac{\textit{h}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\textit{h}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\ 0 & \frac{\textit{h}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & 0 & 0 \\ 0 & -\textit{h}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & 0 & 0 \end{array}\right)\)"
E2,"\(\left(\begin{array}{rrrr} 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & W_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{E2}\left(t, r\right) & X_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{E2}\left(t, r\right) \\ 0 & 0 & X_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{E2}\left(t, r\right) & -W_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{E2}\left(t, r\right) \sin\left(θ\right)^{2} \end{array}\right)\)"
B2,"\(\left(\begin{array}{rrrr} 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & -\frac{X_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{B2}\left(t, r\right)}{\sin\left(θ\right)} & W_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{B2}\left(t, r\right) \sin\left(θ\right) \\ 0 & 0 & W_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{B2}\left(t, r\right) \sin\left(θ\right) & X_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{B2}\left(t, r\right) \sin\left(θ\right) \end{array}\right)\)"
tt,"\(\left(\begin{array}{rrrr} Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{tt}\left(t, r\right) & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \end{array}\right)\)"
Rt,"\(\left(\begin{array}{rrrr} 0 & Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{Rt}\left(t, r\right) & 0 & 0 \\ Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{Rt}\left(t, r\right) & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \end{array}\right)\)"
Et,"\(\left(\begin{array}{rrrr} 0 & 0 & \textit{h}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & \textit{h}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} \\ 0 & 0 & 0 & 0 \\ \textit{h}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & 0 & 0 & 0 \\ \textit{h}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} & 0 & 0 & 0 \end{array}\right)\)"
Bt,"\(\left(\begin{array}{rrrr} 0 & 0 & \frac{\textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\ 0 & 0 & 0 & 0 \\ \frac{\textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & 0 & 0 & 0 \\ -\textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & 0 & 0 & 0 \end{array}\right)\)"


In [20]:
to_table({q:print(latex((hlm[q]*TH[q])[:])) for q in vector_index})

\left(\begin{array}{rrrr}
0 & 0 & \textit{h}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & \textit{h}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} \\
0 & 0 & 0 & 0 \\
\textit{h}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & 0 & 0 & 0 \\
\textit{h}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} & 0 & 0 & 0
\end{array}\right)
\left(\begin{array}{rrrr}
0 & 0 & \frac{\textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\
0 & 0 & 0 & 0 \\
\frac{\textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & 0 & 0 & 0 \\
-\textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & 0 & 0 & 0
\end{array}\right)
\left(\begin{array}{rrrr}
0 & 0 & 0 & 0 \\
0 & 0 & \textit{h}_{lm}^{E1}\left(t, r\right) \frac{\partial\,Y_l^m}{\parti

Et,\(\mathrm{None}\)
Bt,\(\mathrm{None}\)
E1,\(\mathrm{None}\)
B1,\(\mathrm{None}\)


### Polar and Axial perturbations

In [21]:
polar_index = ['L0','T0', 'E1', 'E2','tt','Rt','Et']
axial_index = ['Bt','B1','B2']
h0_pol = sum(hlm[q]*TH[q] for q in polar_index)
h0_ax = sum(hlm[q]*TH[q] for q in axial_index)
to_table(dict([('Axial perturbation:',h0_ax[:]),('Polar perturbation:',h0_pol[:])]))

Axial perturbation:,"\(\left(\begin{array}{rrrr} 0 & 0 & \frac{\textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\ 0 & 0 & \frac{\textit{h}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\textit{h}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\ \frac{\textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & \frac{\textit{h}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\frac{X_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{B2}\left(t, r\right)}{\sin\left(θ\right)} & W_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{B2}\left(t, r\right) \sin\left(θ\right) \\ -\textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & -\textit{h}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & W_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{B2}\left(t, r\right) \sin\left(θ\right) & X_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{B2}\left(t, r\right) \sin\left(θ\right) \end{array}\right)\)"
Polar perturbation:,"\(\left(\begin{array}{rrrr} Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{tt}\left(t, r\right) & Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{Rt}\left(t, r\right) & \textit{h}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & \textit{h}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} \\ Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{Rt}\left(t, r\right) & Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{L0}\left(t, r\right) & \textit{h}_{lm}^{E1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & \textit{h}_{lm}^{E1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} \\ \textit{h}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & \textit{h}_{lm}^{E1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & W_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{E2}\left(t, r\right) + Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{T0}\left(t, r\right) & X_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{E2}\left(t, r\right) \\ \textit{h}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} & \textit{h}_{lm}^{E1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} & X_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{E2}\left(t, r\right) & -W_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{E2}\left(t, r\right) \sin\left(θ\right)^{2} + Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{T0}\left(t, r\right) \sin\left(θ\right)^{2} \end{array}\right)\)"


## Energy-momentum tensor

In [22]:
slm = {k:function(r's'+k,latex_name=r'\textit{s}_{lm}^{'+k+r'}')(t,r) for k in T}
T_pol = sum(slm[q]*TH[q]*8*pi for q in polar_index)
T_ax = sum(slm[q]*TH[q]*8*pi for q in axial_index)
to_table(dict([('Axial EM tensor:',T_ax[:]),('Polar EM tensor:',T_pol[:])]))

Axial EM tensor:,"\(\left(\begin{array}{rrrr} 0 & 0 & \frac{8 \, \pi \textit{s}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -8 \, \pi \textit{s}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\ 0 & 0 & \frac{8 \, \pi \textit{s}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -8 \, \pi \textit{s}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\ \frac{8 \, \pi \textit{s}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & \frac{8 \, \pi \textit{s}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\frac{8 \, \pi X_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{B2}\left(t, r\right)}{\sin\left(θ\right)} & 8 \, \pi W_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{B2}\left(t, r\right) \sin\left(θ\right) \\ -8 \, \pi \textit{s}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & -8 \, \pi \textit{s}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & 8 \, \pi W_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{B2}\left(t, r\right) \sin\left(θ\right) & 8 \, \pi X_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{B2}\left(t, r\right) \sin\left(θ\right) \end{array}\right)\)"
Polar EM tensor:,"\(\left(\begin{array}{rrrr} 8 \, \pi Y_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{tt}\left(t, r\right) & 8 \, \pi Y_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{Rt}\left(t, r\right) & 8 \, \pi \textit{s}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & 8 \, \pi \textit{s}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} \\ 8 \, \pi Y_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{Rt}\left(t, r\right) & 8 \, \pi Y_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{L0}\left(t, r\right) & 8 \, \pi \textit{s}_{lm}^{E1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & 8 \, \pi \textit{s}_{lm}^{E1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} \\ 8 \, \pi \textit{s}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & 8 \, \pi \textit{s}_{lm}^{E1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} & 8 \, \pi W_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{E2}\left(t, r\right) + 8 \, \pi Y_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{T0}\left(t, r\right) & 8 \, \pi X_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{E2}\left(t, r\right) \\ 8 \, \pi \textit{s}_{lm}^{Et}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} & 8 \, \pi \textit{s}_{lm}^{E1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ} & 8 \, \pi X_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{E2}\left(t, r\right) & -8 \, \pi W_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{E2}\left(t, r\right) \sin\left(θ\right)^{2} + 8 \, \pi Y_l^m\left(θ, ϕ\right) \textit{s}_{lm}^{T0}\left(t, r\right) \sin\left(θ\right)^{2} \end{array}\right)\)"


## The Regge-Wheeler gauge

### Axial perturbations

In [23]:
Λlm = function('Λ_lm')(t,r)
ξ_ax0 = Λlm*vector([0,0,-1/sin(θ)*diff(Y,ϕ),sin(θ)*diff(Y,θ)])
ξ_ax = RW.one_form(ξ_ax0, name=r'ξ_{ax}')
show(ξ_ax.display())

ξ_{ax} = -Λ_lm(t, r)*d(Y)/dϕ/sin(θ) dθ + sin(θ)*Λ_lm(t, r)*d(Y)/dθ dϕ

In [24]:
print(latex(ξ_ax.display()))

ξ_{ax} = -\frac{Λ_{{\rm lm}}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} \mathrm{d} θ + \sin\left(θ\right) Λ_{{\rm lm}}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial θ} \mathrm{d} ϕ


In [25]:
DDξ_ax = 2*nabla(ξ_ax).symmetrize()
DDξ_ax.apply_map(lambda q:q.subs(to_XW).full_simplify())
# DDξ_ax[:]

In [26]:
DDξ_ax_check = (- diff(Λlm,t)*T['Bt'] - (diff(Λlm,r)-2/r*Λlm)*T['B1'] + Λlm*T['B2'])
DDξ_ax[:] == DDξ_ax_check

True

In [27]:
h_ax = RW.metric('h_ax', latex_name=r'(h^{\rm{axial}})')
h_ax[:] = (h0_ax - DDξ_ax)[:]
to_table({'h_axial:':h_ax[:]})

h_axial:,"\(\left(\begin{array}{rrrr} 0 & 0 & \frac{\textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} + \frac{\frac{\partial\,Y_l^m}{\partial ϕ} \frac{\partial\,Λ_{\mathit{lm}}}{\partial t}}{\sin\left(θ\right)} & -\textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} - \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \frac{\partial\,Λ_{\mathit{lm}}}{\partial t} \\ 0 & 0 & \frac{\textit{h}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} + \frac{{\left(r \frac{\partial\,Λ_{\mathit{lm}}}{\partial r} - 2 \, Λ_{{\rm lm}}\left(t, r\right)\right)} \frac{\partial\,Y_l^m}{\partial ϕ}}{r \sin\left(θ\right)} & -\textit{h}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} - \frac{{\left(r \frac{\partial\,Λ_{\mathit{lm}}}{\partial r} - 2 \, Λ_{{\rm lm}}\left(t, r\right)\right)} \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ}}{r} \\ \frac{\textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} + \frac{\frac{\partial\,Y_l^m}{\partial ϕ} \frac{\partial\,Λ_{\mathit{lm}}}{\partial t}}{\sin\left(θ\right)} & \frac{\textit{h}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} + \frac{{\left(r \frac{\partial\,Λ_{\mathit{lm}}}{\partial r} - 2 \, Λ_{{\rm lm}}\left(t, r\right)\right)} \frac{\partial\,Y_l^m}{\partial ϕ}}{r \sin\left(θ\right)} & -\frac{X_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{B2}\left(t, r\right)}{\sin\left(θ\right)} + \frac{X_l^m\left(θ, ϕ\right) Λ_{{\rm lm}}\left(t, r\right)}{\sin\left(θ\right)} & W_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{B2}\left(t, r\right) \sin\left(θ\right) - W_l^m\left(θ, ϕ\right) \sin\left(θ\right) Λ_{{\rm lm}}\left(t, r\right) \\ -\textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} - \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \frac{\partial\,Λ_{\mathit{lm}}}{\partial t} & -\textit{h}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} - \frac{{\left(r \frac{\partial\,Λ_{\mathit{lm}}}{\partial r} - 2 \, Λ_{{\rm lm}}\left(t, r\right)\right)} \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ}}{r} & W_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{B2}\left(t, r\right) \sin\left(θ\right) - W_l^m\left(θ, ϕ\right) \sin\left(θ\right) Λ_{{\rm lm}}\left(t, r\right) & X_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{B2}\left(t, r\right) \sin\left(θ\right) - X_l^m\left(θ, ϕ\right) \sin\left(θ\right) Λ_{{\rm lm}}\left(t, r\right) \end{array}\right)\)"


In [28]:
gauge_ax = solve(h_ax[3,3].expr(),Λlm,solution_dict=True)[0]
to_table(gauge_ax)

"\(Λ_{{\rm lm}}\left(t, r\right)\)","\(\textit{h}_{lm}^{B2}\left(t, r\right)\)"


In [29]:
gauge_ax2 = {diff(hlm['B2'],k):0 for k in [0,t,r]}
h_ax.apply_map(lambda q:q.substitute_function(gauge_ax).subs(gauge_ax2).expand().factor())
to_table({'h_axial (gauged):':h_ax[:]})

h_axial (gauged):,"\(\left(\begin{array}{rrrr} 0 & 0 & \frac{\textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\ 0 & 0 & \frac{\textit{h}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\textit{h}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\ \frac{\textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & \frac{\textit{h}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & 0 & 0 \\ -\textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & -\textit{h}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & 0 & 0 \end{array}\right)\)"


In [30]:
print(latex(h_ax[:]))

\left(\begin{array}{rrrr}
0 & 0 & \frac{\textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\
0 & 0 & \frac{\textit{h}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\textit{h}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\
\frac{\textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & \frac{\textit{h}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & 0 & 0 \\
-\textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & -\textit{h}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & 0 & 0
\end{array}\right)


### Polar perturbations

In [31]:
ξ = {q:function('ξ'+q, latex_name=r'ξ_{lm}^{'+q+'}')(t,r) for q in ['t','R','E']}
ξ_pol0 = vector([ξ['t']*Y,ξ['R']*Y,0,0]) + ξ['E']*vector([0,0,diff(Y,θ),diff(Y,ϕ)])
ξ_pol = RW.one_form(ξ_pol0, name=r'ξ_{pol}')
show(ξ_pol.display())

ξ_{pol} = Y(θ, ϕ)*ξt(t, r) dt + Y(θ, ϕ)*ξR(t, r) dr + ξE(t, r)*d(Y)/dθ dθ + ξE(t, r)*d(Y)/dϕ dϕ

In [32]:
DDξ_pol = 2*nabla(ξ_pol).symmetrize()

In [33]:
h_pol = RW.metric('h_pol', latex_name=r'(h^{\rm{polar}})')
h_pol[:] = (h0_pol - DDξ_pol)[:]
h_pol.apply_map(lambda q:q.expand().substitute_function(from_XW).full_simplify())
to_table({'h_polar:':h_pol[:]})

h_polar:


In [34]:
gauge_pol = [solve(h_pol[3,q[0]].expr(),ξ[q[1]],solution_dict=True)[0] for q in enumerate(['t','R','E'])]
for q in ['E2','E1','Et']: gauge_pol.append({diff(hlm[q],k):0 for k in [0,t,r]})
gauge_pol.append({diff(hlm['E2'],*k):0 for k in [(t,t),(t,r),(r,r)]})
for k in gauge_pol:
    show(to_table(k))

"\(ξ_{lm}^{t}\left(t, r\right)\)","\(\textit{h}_{lm}^{Et}\left(t, r\right) - \frac{\partial}{\partial t}ξ_{lm}^{E}\left(t, r\right)\)"


"\(ξ_{lm}^{R}\left(t, r\right)\)","\(\frac{r \textit{h}_{lm}^{E1}\left(t, r\right) - r \frac{\partial}{\partial r}ξ_{lm}^{E}\left(t, r\right) + 2 \, ξ_{lm}^{E}\left(t, r\right)}{r}\)"


"\(ξ_{lm}^{E}\left(t, r\right)\)","\(\textit{h}_{lm}^{E2}\left(t, r\right)\)"


"\(\textit{h}_{lm}^{E2}\left(t, r\right)\)",\(0\)
"\(\frac{\partial}{\partial t}\textit{h}_{lm}^{E2}\left(t, r\right)\)",\(0\)
"\(\frac{\partial}{\partial r}\textit{h}_{lm}^{E2}\left(t, r\right)\)",\(0\)


"\(\textit{h}_{lm}^{E1}\left(t, r\right)\)",\(0\)
"\(\frac{\partial}{\partial t}\textit{h}_{lm}^{E1}\left(t, r\right)\)",\(0\)
"\(\frac{\partial}{\partial r}\textit{h}_{lm}^{E1}\left(t, r\right)\)",\(0\)


"\(\textit{h}_{lm}^{Et}\left(t, r\right)\)",\(0\)
"\(\frac{\partial}{\partial t}\textit{h}_{lm}^{Et}\left(t, r\right)\)",\(0\)
"\(\frac{\partial}{\partial r}\textit{h}_{lm}^{Et}\left(t, r\right)\)",\(0\)


"\(\frac{\partial^{2}}{(\partial t)^{2}}\textit{h}_{lm}^{E2}\left(t, r\right)\)",\(0\)
"\(\frac{\partial^{2}}{\partial t\partial r}\textit{h}_{lm}^{E2}\left(t, r\right)\)",\(0\)
"\(\frac{\partial^{2}}{(\partial r)^{2}}\textit{h}_{lm}^{E2}\left(t, r\right)\)",\(0\)


In [35]:
for k in gauge_pol[:3]: h_pol.apply_map(lambda q:q.substitute_function(k))
for k in gauge_pol[3:]: h_pol.apply_map(lambda q:q.subs(k))
to_table({'h_polar (gauged):':h_pol[:]})

h_polar (gauged):,"\(\left(\begin{array}{rrrr} Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{tt}\left(t, r\right) & Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{Rt}\left(t, r\right) & 0 & 0 \\ Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{Rt}\left(t, r\right) & Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{L0}\left(t, r\right) & 0 & 0 \\ 0 & 0 & Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{T0}\left(t, r\right) & 0 \\ 0 & 0 & 0 & Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{T0}\left(t, r\right) \sin\left(θ\right)^{2} \end{array}\right)\)"


### General form

In [36]:
ep = var('epsilon')
ax = var('ax')
h_pert = ep*(h_pol + ax*h_ax)
g_pert_0 = RW.metric('g_pert', latex_name=(r'(g_{\mathrm{perturbed}})'))
g_pert_0[:] = (g + h_pert)[:]
to_table({'g + h_axial + h_polar:':g_pert_0[:]})

g + h_axial + h_polar:,"\(\left(\begin{array}{rrrr} \epsilon Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{tt}\left(t, r\right) + \frac{2 \, M}{r} - 1 & \epsilon Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{Rt}\left(t, r\right) & \frac{\mathit{ax} \epsilon \textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\mathit{ax} \epsilon \textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\ \epsilon Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{Rt}\left(t, r\right) & \epsilon Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{L0}\left(t, r\right) - \frac{1}{\frac{2 \, M}{r} - 1} & \frac{\mathit{ax} \epsilon \textit{h}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\mathit{ax} \epsilon \textit{h}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\ \frac{\mathit{ax} \epsilon \textit{h}_{lm}^{Bt}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & \frac{\mathit{ax} \epsilon \textit{h}_{lm}^{B1}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & \epsilon Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{T0}\left(t, r\right) + r^{2} & 0 \\ -\mathit{ax} \epsilon \textit{h}_{lm}^{Bt}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & -\mathit{ax} \epsilon \textit{h}_{lm}^{B1}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & 0 & \epsilon Y_l^m\left(θ, ϕ\right) \textit{h}_{lm}^{T0}\left(t, r\right) \sin\left(θ\right)^{2} + r^{2} \sin\left(θ\right)^{2} \end{array}\right)\)"


In [37]:
new_functions = {hlm['tt']:-g[0,0].expr()*function('H0', latex_name=r'H_{lm}^{(0)}')(t,r),
                 hlm['L0']:g[1,1].expr()*function('H2', latex_name=r'H_{lm}^{(2)}')(t,r),
                 hlm['T0']:g[2,2].expr()*function('K', latex_name=r'K_{lm}')(t,r),
                 hlm['Rt']:function('H1', latex_name=r'H_{lm}^{(1)}')(t,r),
                 hlm['Bt']:-1*function('h0', latex_name=r'h_{lm}^{(0)}')(t,r),
                 hlm['B1']:-1*function('h1', latex_name=r'h_{lm}^{(1)}')(t,r)
                }
to_table(new_functions)

"\(\textit{h}_{lm}^{tt}\left(t, r\right)\)","\(-{\left(\frac{2 \, M}{r} - 1\right)} H_{lm}^{(0)}\left(t, r\right)\)"
"\(\textit{h}_{lm}^{L0}\left(t, r\right)\)","\(-\frac{H_{lm}^{(2)}\left(t, r\right)}{\frac{2 \, M}{r} - 1}\)"
"\(\textit{h}_{lm}^{T0}\left(t, r\right)\)","\(r^{2} K_{lm}\left(t, r\right)\)"
"\(\textit{h}_{lm}^{Rt}\left(t, r\right)\)","\(H_{lm}^{(1)}\left(t, r\right)\)"
"\(\textit{h}_{lm}^{Bt}\left(t, r\right)\)","\(-h_{lm}^{(0)}\left(t, r\right)\)"
"\(\textit{h}_{lm}^{B1}\left(t, r\right)\)","\(-h_{lm}^{(1)}\left(t, r\right)\)"


In [38]:
h_pert.apply_map(lambda q: q.subs(new_functions))
to_table({'g + h_axial + h_polar:':(g+h_pert)[:]})

g + h_axial + h_polar:,"\(\left(\begin{array}{rrrr} -\epsilon {\left(\frac{2 \, M}{r} - 1\right)} H_{lm}^{(0)}\left(t, r\right) Y_l^m\left(θ, ϕ\right) + \frac{2 \, M}{r} - 1 & \epsilon H_{lm}^{(1)}\left(t, r\right) Y_l^m\left(θ, ϕ\right) & -\frac{\mathit{ax} \epsilon h_{lm}^{(0)}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & \mathit{ax} \epsilon h_{lm}^{(0)}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\ \epsilon H_{lm}^{(1)}\left(t, r\right) Y_l^m\left(θ, ϕ\right) & -\frac{\epsilon H_{lm}^{(2)}\left(t, r\right) Y_l^m\left(θ, ϕ\right)}{\frac{2 \, M}{r} - 1} - \frac{1}{\frac{2 \, M}{r} - 1} & -\frac{\mathit{ax} \epsilon h_{lm}^{(1)}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & \mathit{ax} \epsilon h_{lm}^{(1)}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} \\ -\frac{\mathit{ax} \epsilon h_{lm}^{(0)}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & -\frac{\mathit{ax} \epsilon h_{lm}^{(1)}\left(t, r\right) \frac{\partial\,Y_l^m}{\partial ϕ}}{\sin\left(θ\right)} & \epsilon r^{2} K_{lm}\left(t, r\right) Y_l^m\left(θ, ϕ\right) + r^{2} & 0 \\ \mathit{ax} \epsilon h_{lm}^{(0)}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & \mathit{ax} \epsilon h_{lm}^{(1)}\left(t, r\right) \sin\left(θ\right) \frac{\partial\,Y_l^m}{\partial θ} & 0 & \epsilon r^{2} K_{lm}\left(t, r\right) Y_l^m\left(θ, ϕ\right) \sin\left(θ\right)^{2} + r^{2} \sin\left(θ\right)^{2} \end{array}\right)\)"


### RW gauge checker

In [39]:
show(h_pert[2,3]==0)
show(h_pert[3,3]==sin(θ)^2*h_pert[2,2])

True

True

In [40]:
for q in [0,1]:
    show(diff(h_pert[q,3],ϕ)==-sin(θ)*diff(h_pert[q,2]*sin(θ),θ))

True

True

In [41]:
stage_time()

Total time elapsed until the end of stage 0: 00:00:55


## Field equations

$$(G_\mathrm{linear})_{\alpha\beta} = \frac12\left(\nabla^\mu\nabla_\beta \bar{h}_{\alpha\mu} + \nabla^\mu\nabla_\alpha \bar{h}_{\beta\mu} - \square\bar{h}_{\mu\nu} - g_{\alpha\beta}\nabla^\mu\nabla^\nu \bar{h}_{\mu\nu}\right)$$
$$\bar{h}_{\alpha\beta} = {h}_{\alpha\beta}-\frac12 g_{\alpha\beta}h$$

In [42]:
h_ax.apply_map(lambda q: q.subs(new_functions))
# h_ax = h_pert
hb_ax = h_ax - 1/2*g*(h_ax.up(g,0).trace())

In [43]:
%time G_dal = hb_ax.dalembertian(g)

CPU times: user 274 ms, sys: 11.2 ms, total: 285 ms
Wall time: 284 ms


In [44]:
%time G_div = g*hb_ax.div(g).div(g)

CPU times: user 161 ms, sys: 850 µs, total: 162 ms
Wall time: 142 ms


In [45]:
%time G_sym = (nabla(nabla(hb_ax)).up(g,3)['_bac^a']).symmetrize()

CPU times: user 122 ms, sys: 3.81 ms, total: 126 ms
Wall time: 126 ms


In [46]:
%time G = G_sym - 1/2*(G_dal + G_div)
# G.set_name('G')
# show(G.display_comp(only_nonredundant=True))

CPU times: user 1.59 ms, sys: 0 ns, total: 1.59 ms
Wall time: 1.6 ms


In [47]:
stage_time()

Total time elapsed until the end of stage 1: 00:00:57


In [48]:
Gp = G.copy()
Gp.set_name('(Gp)')
# G.apply_map(lambda q: q.subs(to_XW).expand().trig_simplify().factor())
for q in (2,3):
    for k in (2,3):
        Gp[q,k] = Gp[q,k].expr().subs(to_XW).expand().trig_simplify().factor()
# show(Gp.display_comp(only_nonredundant=True))

In [49]:
# G_θϕ = (-2*G[2,3]/(W*sin(θ))).expand()
# show(G_θϕ)
sol_h0 = solve(G[2,3].expr(),diff(h0(t,r),t), solution_dict=True)[0]
for q in (2,3):
    for k in (2,3):
        Gp[q,k] = Gp[q,k].expr().subs(sol_h0).expand().trig_simplify().factor()
sol_h0.update({diff(list(sol_h0.keys())[0],r):diff(list(sol_h0.values())[0],r)})
to_table(sol_h0)

"\(\frac{\partial}{\partial t}h_{lm}^{(0)}\left(t, r\right)\)","\(-\frac{2 \, {\left(2 \, M^{2} - M r\right)} h_{lm}^{(1)}\left(t, r\right) - {\left(4 \, M^{2} r - 4 \, M r^{2} + r^{3}\right)} \frac{\partial}{\partial r}h_{lm}^{(1)}\left(t, r\right)}{r^{3}}\)"
"\(\frac{\partial^{2}}{\partial t\partial r}h_{lm}^{(0)}\left(t, r\right)\)","\(\frac{2 \, M h_{lm}^{(1)}\left(t, r\right) + {\left(4 \, M^{2} - 8 \, M r + 3 \, r^{2}\right)} \frac{\partial}{\partial r}h_{lm}^{(1)}\left(t, r\right) - 2 \, {\left(2 \, M^{2} - M r\right)} \frac{\partial}{\partial r}h_{lm}^{(1)}\left(t, r\right) + {\left(4 \, M^{2} r - 4 \, M r^{2} + r^{3}\right)} \frac{\partial^{2}}{(\partial r)^{2}}h_{lm}^{(1)}\left(t, r\right)}{r^{3}} + \frac{3 \, {\left(2 \, {\left(2 \, M^{2} - M r\right)} h_{lm}^{(1)}\left(t, r\right) - {\left(4 \, M^{2} r - 4 \, M r^{2} + r^{3}\right)} \frac{\partial}{\partial r}h_{lm}^{(1)}\left(t, r\right)\right)}}{r^{4}}\)"


In [50]:
to_sph2 = {diff(Y,ϕ,3):Y*(diff(sph,ϕ,3)/sph).expand(),
           diff(Y,ϕ,ϕ,θ):diff(Y,θ)*(diff(sph,ϕ,ϕ,θ)/diff(sph,θ)).expand(),
           diff(Y,ϕ,2):Y*(diff(sph,ϕ,ϕ)/sph).expand(),
           diff(Y,ϕ,θ):diff(Y,θ)*(diff(sph,ϕ,θ)/diff(sph,θ)).expand(),
           diff(Y,θ,θ,ϕ):diff(Y,θ,2)*(diff(sph,ϕ)/sph).expand(),
           diff(Y,ϕ):Y*(diff(sph,ϕ)/sph).expand()
          }
to_table(to_sph2)

"\(\frac{\partial^{3}}{(\partial ϕ)^{3}}Y_l^m\left(θ, ϕ\right)\)","\(-i \, m^{3} Y_l^m\left(θ, ϕ\right)\)"
"\(\frac{\partial^{3}}{\partial θ(\partial ϕ)^{2}}Y_l^m\left(θ, ϕ\right)\)","\(-m^{2} \frac{\partial}{\partial θ}Y_l^m\left(θ, ϕ\right)\)"
"\(\frac{\partial^{2}}{(\partial ϕ)^{2}}Y_l^m\left(θ, ϕ\right)\)","\(-m^{2} Y_l^m\left(θ, ϕ\right)\)"
"\(\frac{\partial^{2}}{\partial θ\partial ϕ}Y_l^m\left(θ, ϕ\right)\)","\(i \, m \frac{\partial}{\partial θ}Y_l^m\left(θ, ϕ\right)\)"
"\(\frac{\partial^{3}}{(\partial θ)^{2}\partial ϕ}Y_l^m\left(θ, ϕ\right)\)","\(i \, m \frac{\partial^{2}}{(\partial θ)^{2}}Y_l^m\left(θ, ϕ\right)\)"
"\(\frac{\partial}{\partial ϕ}Y_l^m\left(θ, ϕ\right)\)","\(i \, m Y_l^m\left(θ, ϕ\right)\)"


In [51]:
Gp.apply_map(lambda q: q.subs(to_sph2).expand())
# show(Gp.display_comp(only_nonredundant=True))

### Vacuum equations

In [52]:
# sourceless
eq_B2 = solve(G[2,3].expr(),diff(h0(t,r),t))[0]
# sourced
# eq_B2 = (solve(G[2,3].expr()==T_ax[2,3].expr(),slm['B2'])[0].subs(from_XW).full_simplify()*-16*pi).expand()
disp(eq_B2) #eq 12.93

d(h0)/dt == -(2*(2*M^2 - M*r)*h1(t, r) - (4*M^2*r - 4*M*r^2 + r^3)*d(h1)/dr)/r^3

In [53]:
print(latex(eq_B2))

\frac{\partial}{\partial t}h_{lm}^{(0)}\left(t, r\right) = -\frac{2 \, {\left(2 \, M^{2} - M r\right)} h_{lm}^{(1)}\left(t, r\right) - {\left(4 \, M^{2} r - 4 \, M r^{2} + r^{3}\right)} \frac{\partial}{\partial r}h_{lm}^{(1)}\left(t, r\right)}{r^{3}}


In [54]:
# sourceless
eq_Bt0 = (Gp[0,2].expr().subs(sol_h0).subs(lap_eq)).solve(diff(h0(t,r),r,2))[0].expand()
eq_Bt = eq_Bt0.lhs() == mycollect(mycollect(mycollect(eq_Bt0.rhs(),h0(t,r)),diff(h1(t,r),t)),diff(h1(t,r),r,t))
# sourced
# eq_Bt0 = ((Gp[0,2].expr().subs(sol_h0).subs(lap_eq)==T_ax[0,2].expr()).solve(slm['Bt'])[0].subs(to_sph2).full_simplify()*16*pi/f).expand()
# eq_Bt = eq_Bt0.lhs() == mycollect(eq_Bt0.rhs(),h0(t,r))
disp(eq_Bt) #eq 12.91

d^2(h0)/dr^2 == -(cl*r/(2*M*r^2 - r^3) - 4*M/(2*M*r^2 - r^3))*h0(t, r) + 2*(2*M*r/(2*M*r^2 - r^3) - r^2/(2*M*r^2 - r^3))*d(h1)/dt + (2*M*r^2/(2*M*r^2 - r^3) - r^3/(2*M*r^2 - r^3))*d^2(h1)/dtdr

In [55]:
# sourceless
eq_B1_0 = (Gp[1,2].expr().subs(lap_eq)).solve(diff(h1(t,r),t,2))[0].expand()
eq_B1_1 = eq_B1_0.lhs() == mycollect(eq_B1_0.rhs(),h1(t,r))
# sourced
# eq_B1_0 = ((Gp[1,2].expr().subs(lap_eq)==T_ax[1,2].expr()).solve(slm['B1'])[0].subs(to_sph2).full_simplify()*-16*pi*f).expand()
# eq_B1_1 = eq_B1_0.lhs() == mycollect(eq_B1_0.rhs(),h1(t,r))
disp(eq_B1_1) #eq 12.92 l is different

d^2(h1)/dt^2 == (2*M*cl/r^3 - cl/r^2 - 4*M/r^3 + 2/r^2)*h1(t, r) - 2*d(h0)/dt/r + d^2(h0)/dtdr

In [56]:
print(latex(eq_B1_1))

\frac{\partial^{2}}{(\partial t)^{2}}h_{lm}^{(1)}\left(t, r\right) = {\left(\frac{2 \, M {l(l+1)}}{r^{3}} - \frac{{l(l+1)}}{r^{2}} - \frac{4 \, M}{r^{3}} + \frac{2}{r^{2}}\right)} h_{lm}^{(1)}\left(t, r\right) - \frac{2 \, \frac{\partial}{\partial t}h_{lm}^{(0)}\left(t, r\right)}{r} + \frac{\partial^{2}}{\partial t\partial r}h_{lm}^{(0)}\left(t, r\right)


Or (using eq. B2)

In [57]:
eq_B1 = eq_B1_1.lhs() == mycollect(mycollect(mycollect(eq_B1_1.rhs().subs(sol_h0),h1(t,r)),diff(h1(t,r),r)),diff(h1(t,r),r,2))
disp(eq_B1)

d^2(h1)/dt^2 == (2*M*cl/r^3 - cl/r^2 + 20*M^2/r^4 - 12*M/r^3 + 2/r^2)*h1(t, r) - 2*(10*M^2/r^3 - 7*M/r^2 + 1/r)*d(h1)/dr + (4*M^2/r^2 - 4*M/r + 1)*d^2(h1)/dr^2

In [58]:
stage_time()

Total time elapsed until the end of stage 2: 00:01:02


### Regge-Wheeler equation

In [59]:
ω = var('ω')
Q(r) = function('Q', latex_name=r'Q_{lm}')(r,ω)
fun_Q = {h1(t,r):r*Q*exp(-i*ω*t)/f}
RW0 = (eq_B1.substitute_function(fun_Q)*exp(i*ω*t)).expand().canonicalize_radical()
show(RW0)

r^2*ω^2*Q(r, ω)/(2*M - r) == (2*M*r*diff(Q(r, ω), r) - (cl*r - 6*M)*Q(r, ω) - (2*M*r^2 - r^3)*diff(Q(r, ω), r, r))/r^2

In [60]:
from de_tools import change_variable
ftortoise = function('ftortoise', latex_name=r'r_\star')(r)
tortoise = var('tortoise', latex_name=r'r_\star', domain='real')
dtortoise = sqrt((-g[1,1]/g[0,0]).expr()).canonicalize_radical()
RW1 = change_variable(RW0, Q, r, ftortoise, tortoise, dtortoise, expand=True)
disp(RW1)

canonicalize_de for Q(tortoise, ω): passed!


(r^4*ω^2 + 2*M*cl*r - cl*r^2 - 12*M^2 + 6*M*r)*Q(tortoise, ω)/r^4 + d^2(Q)/dtortoise^2 == 0

$$\dfrac{d^2 Q_{lm}(r,\omega)}{dr_\star} + \left[\omega^2 - V_l(r)\right]Q_{lm}(r,\omega) = 0$$

In [61]:
from de_tools import get_de_coefficients
V(r,l) = ω^2-get_de_coefficients(RW1, Q(tortoise), tortoise)[0].expand().subs(sep_const)
show(LatexExpr(r'V_l(r) = '),mycollect(V(r,l),f))

V_l(r) =  -2*M*(l + 1)*l/r^3 + (l + 1)*l/r^2 + 12*M^2/r^4 - 6*M/r^3

In [62]:
V_schw = -2*M*(l + 1)*l/r^3 + (l + 1)*l/r^2 + 12*M^2/r^4 - 6*M/r^3
show(LatexExpr(r'V_l(r)\Big|_{\rm{Schwarzschild}} = '),V_schw)
show(LatexExpr(r'V_l(r)\Big|_{\rm{Schwarzschild}} = '),(1-2*M/r)*(l*(l+1)/r^2-6*M/r^3))

V_l(r)\Big|_{\rm{Schwarzschild}} =  -2*M*(l + 1)*l/r^3 + (l + 1)*l/r^2 + 12*M^2/r^4 - 6*M/r^3

V_l(r)\Big|_{\rm{Schwarzschild}} =  -((l + 1)*l/r^2 - 6*M/r^3)*(2*M/r - 1)

Checking the equation at https://bhptoolkit.org/ReggeWheeler/ (from RW0)

In [63]:
U = var('U_l')
lV = (-U + V(r,l).subs(solve(l*(l+1)==cl,l)[0]).expand()).solve(cl,solution_dict=True)[0]
from de_tools import canonicalize_de
RW_check = (RW0.solve(diff(Q,r,2))[0]*(-1)*f^2).expand().subs(lV)
disp(canonicalize_de(RW_check,Q(r,ω),r))

canonicalize_de for Q(r, ω): passed!


(ω^2 - U_l)*r^2*Q(r, ω)/(2*M - r)^2 - 2*M*d(Q)/dr/((2*M - r)*r) + d^2(Q)/dr^2 == 0

In [64]:
stage_time()

Total time elapsed until the end of stage 3: 00:01:04


***

## The Zerilli equation

In [65]:
h_pol.apply_map(lambda q: q.subs(new_functions))
# h_pol = h_pert
hb_pol = h_pol - 1/2*g*(h_pol.up(g,0).trace())

In [66]:
%time G_dal_pol = hb_pol.dalembertian(g)

CPU times: user 126 ms, sys: 0 ns, total: 126 ms
Wall time: 126 ms


In [67]:
%time G_div_pol = g*hb_pol.div(g).div(g)

CPU times: user 1.68 s, sys: 19.4 ms, total: 1.7 s
Wall time: 1.41 s


In [68]:
%time G_sym_pol = (nabla(nabla(hb_pol)).up(g,3)['_bac^a']).symmetrize()

CPU times: user 138 ms, sys: 140 µs, total: 138 ms
Wall time: 137 ms


In [69]:
%time G_pol = G_sym_pol - 1/2*(G_dal_pol + G_div_pol)

CPU times: user 1.94 ms, sys: 122 µs, total: 2.07 ms
Wall time: 2.08 ms


In [70]:
stage_time()

Total time elapsed until the end of stage 4: 00:01:07


In [71]:
Gp_pol = G_pol.copy()
Gp_pol.set_name('(Gp)')
for q in (2,3):
    for k in (2,3):
        Gp_pol[q,k] = Gp_pol[q,k].expr().subs(to_XW).expand().trig_simplify().factor()
# show(Gp_pol.display_comp(only_nonredundant=True))

### Vacuum equations

A partir daqui começará a resolução das 7 equação diferenciais para a solução de vácuo, para a perturbação polar. Começaremos resolvendo a primeira equação, da componente `Gp_pol[2, 3]`, de acordo com a equação (12.371):

In [72]:
sol_H2 = solve(Gp_pol[2,3].expr(),H2(t,r), solution_dict=True)[0] # eq. 12.371

for q in (2,3):
    for k in (2,3):
        Gp_pol[q,k] = Gp_pol[q,k].expr().subs(sol_H2).expand().trig_simplify().factor()
sol_H2.update({diff(list(sol_H2.keys())[0],r):diff(list(sol_H2.values())[0],r)})
sol_H2.update({diff(list(sol_H2.keys())[0],r, 2):diff(list(sol_H2.values())[0],r,2)})
sol_H2.update({diff(list(sol_H2.keys())[0],t):diff(list(sol_H2.values())[0],t)})
sol_H2.update({diff(list(sol_H2.keys())[0],t,2):diff(list(sol_H2.values())[0],t,2)})
to_table(sol_H2)

"\(H_{lm}^{(2)}\left(t, r\right)\)","\(H_{lm}^{(0)}\left(t, r\right)\)"
"\(\frac{\partial}{\partial r}H_{lm}^{(2)}\left(t, r\right)\)","\(\frac{\partial}{\partial r}H_{lm}^{(0)}\left(t, r\right)\)"
"\(\frac{\partial^{2}}{(\partial r)^{2}}H_{lm}^{(2)}\left(t, r\right)\)","\(\frac{\partial^{2}}{(\partial r)^{2}}H_{lm}^{(0)}\left(t, r\right)\)"
"\(\frac{\partial}{\partial t}H_{lm}^{(2)}\left(t, r\right)\)","\(\frac{\partial}{\partial t}H_{lm}^{(0)}\left(t, r\right)\)"
"\(\frac{\partial^{2}}{(\partial t)^{2}}H_{lm}^{(2)}\left(t, r\right)\)","\(\frac{\partial^{2}}{(\partial t)^{2}}H_{lm}^{(0)}\left(t, r\right)\)"


Agora, injetaremos o resultado acima em `Gp_pol`, para substituir $H^{(2)}$ por $H^{(0)}$:

In [73]:
Gp_pol.apply_map(lambda q: q.subs(to_sph2).subs(sol_H2).expand())
# show(Gp_pol.display_comp(only_nonredundant=True))

A segunda equação de vácuo será resolvida utilizando `Gp_pol[0,0]`, de acordo com a equação (12.365):

In [74]:
eq_tt_1 = (Gp_pol[0,0].expr().subs(lap_eq)).solve(diff(K(t,r),r,2))[0].full_simplify().expand()
eq_tt = eq_tt_1.lhs() == mycollect(mycollect(mycollect(mycollect(eq_tt_1.rhs(), H0(t, r)), 
                        K(t, r)), diff(H0(t, r), r)), diff(K(t, r), r)) # eq. 12.365

In [75]:
eq_tt

diff(K(t, r), r, r) == -1/2*(cl/(2*M*r - r^2) + 2/(2*M*r - r^2))*H0(t, r) - 1/2*(cl/(2*M*r - r^2) - 2/(2*M*r - r^2))*K(t, r) + (2*M/(2*M*r - r^2) - r/(2*M*r - r^2))*diff(H0(t, r), r) - (5*M/(2*M*r - r^2) - 3*r/(2*M*r - r^2))*diff(K(t, r), r)

A terceira equação de vácuo será resolvida utilizando `Gp_pol[1,0]`, de acordo com a equação (12.366):

In [76]:
eq_Rt_1 = (Gp_pol[1,0].expr().subs(lap_eq)).solve(diff(K(t,r),r,t))[0].full_simplify().expand()
eq_Rt = eq_Rt_1.lhs() == mycollect(mycollect(mycollect(eq_Rt_1.rhs(), diff(H0(t, r), t)), diff(K(t, r), t)), H1(t, r)) # eq. 12.366

In [77]:
eq_Rt

diff(K(t, r), t, r) == 1/2*(2*M*cl/(2*M*r^2 - r^3) - cl*r/(2*M*r^2 - r^3))*H1(t, r) + (2*M*r/(2*M*r^2 - r^3) - r^2/(2*M*r^2 - r^3))*diff(H0(t, r), t) - (3*M*r/(2*M*r^2 - r^3) - r^2/(2*M*r^2 - r^3))*diff(K(t, r), t)

A quarta equação de vácuo será resolvida utilizando `Gp_pol[1,1]`, de acordo com a equação (12.367):

In [78]:
eq_L0_1 = (Gp_pol[1,1].expr().subs(lap_eq)).solve(diff(K(t,r),t, 2))[0].expand()
eq_L0 = eq_L0_1.lhs() == mycollect(mycollect(mycollect(mycollect(mycollect(eq_L0_1.rhs(), diff(H0(t, r), r)), 
                        diff(K(t, r), r)), K(t, r)), H0(t, r)), diff(H1(t, r), t)) # eq. 12.367

In [79]:
eq_L0

diff(K(t, r), t, t) == -1/2*(2*M*cl/r^3 - cl/r^2 - 4*M/r^3 + 2/r^2)*H0(t, r) + 1/2*(2*M*cl/r^3 - cl/r^2 - 4*M/r^3 + 2/r^2)*K(t, r) - (4*M^2/r^3 - 4*M/r^2 + 1/r)*diff(H0(t, r), r) - 2*(2*M/r^2 - 1/r)*diff(H1(t, r), t) + (2*M^2/r^3 - 3*M/r^2 + 1/r)*diff(K(t, r), r)

A quinta equação de vácuo será resolvida utilizando `Gp_pol[2,2]`, de acordo com a equação (12.368):

In [80]:
eq_T0_1 = (Gp_pol[2,2].expr().subs(lap_eq)).solve(diff(K(t,r),t, 2))[0].simplify().expand()
eq_T0 = eq_T0_1.lhs() == mycollect(mycollect(mycollect(mycollect(mycollect(mycollect(eq_T0_1.rhs(), diff(K(t, r), r, 2)), diff(H0(t, r), r, 2)),
                        diff(H1(t, r), r, t)), diff(K(t, r), r)), diff(H0(t, r), r)), diff(H1(t, r), t))# eq. 12.368

In [81]:
eq_T0

diff(K(t, r), t, t) == 2*(2*M/r^2 - 1/r)*diff(H0(t, r), r) - (4*M^2/r^2 - 4*M/r + 1)*diff(H0(t, r), r, r) - 2*(M/r^2 - 1/r)*diff(H1(t, r), t) - 2*(2*M/r - 1)*diff(H1(t, r), t, r) + 2*(2*M^2/r^3 - 3*M/r^2 + 1/r)*diff(K(t, r), r) + (4*M^2/r^2 - 4*M/r + 1)*diff(K(t, r), r, r) - diff(H0(t, r), t, t)

A sexta equação de vácuo será resolvida utilizando `Gp_pol[0,2]`, de acordo com a equação (12.369):

In [82]:
eq_Et_1 = (Gp_pol[0,2].expr().subs(lap_eq)).solve(diff(K(t,r),t))[0].simplify().expand()
eq_Et= eq_Et_1.lhs() == mycollect(eq_Et_1.rhs(), diff(H1(t, r), r))
# eq. 12.369

In [83]:
show(eq_Et)

diff(K(t, r), t) == -(2*M/r - 1)*diff(H1(t, r), r) + 2*M*H1(t, r)/r^2 - diff(H0(t, r), t)

Por fim, a setima equação de vácuo será resolvida utilizando `Gp_pol[1,2]`, de acordo com a equação (12.370):

In [84]:
eq_E1_1 = (Gp_pol[1,2].expr().subs(lap_eq)).solve(diff(H1(t,r),t))[0].simplify().expand()
eq_E1= eq_E1_1.lhs() == mycollect(mycollect(eq_E1_1.rhs(), diff(K(t, r), r)), diff(H0(t, r), r)) # eq. 12.370

In [85]:
eq_E1

diff(H1(t, r), t) == -(2*M/r - 1)*diff(H0(t, r), r) + (2*M/r - 1)*diff(K(t, r), r) + 2*M*H0(t, r)/r^2

Para resolver as 6 equações restantes acima, é conveniente realizar uma tranformada de Fourier com respeito ao tempo. Faremos a transformada como se segue:

In [86]:
Kf(r,ω) = function('Kf', latex_name=r'\tilde{K}_{lm}')(r,ω)
H0f(r,ω) = function('H0f', latex_name=r'\tilde{H}^{(0)}_{lm}')(r,ω)
H1f(r,ω) = function('H1f', latex_name=r'\tilde{H}^{(1)}_{lm}')(r,ω)
fun_H = {K(t,r):Kf*exp(-i*ω*t), H0(t,r):H0f*exp(-i*ω*t), H1(t,r):H1f*exp(-i*ω*t)}
show(fun_H)

{K(t, r): (r, ω) |--> Kf(r, ω)*e^(-I*t*ω),
 H0(t, r): (r, ω) |--> H0f(r, ω)*e^(-I*t*ω),
 H1(t, r): (r, ω) |--> H1f(r, ω)*e^(-I*t*ω)}

Aplicando a tranformada na equação (12.366), temos a equação (12.372):

In [87]:
eq_Z0_Rt = (eq_Rt.substitute_function(fun_H)*exp(i*ω*t))/(-i*ω)

In [88]:
eq_Z0_K = eq_Z0_Rt.lhs() == mycollect(mycollect(mycollect(eq_Z0_Rt.rhs(), Kf(r, ω)), H0f(r, ω)), H1f(r, ω))
# eq. 12.372

In [89]:
eq_Z0_K

diff(Kf(r, ω), r) == (2*M*r/(2*M*r^2 - r^3) - r^2/(2*M*r^2 - r^3))*H0f(r, ω) - 1/2*(-2*I*M*cl/((2*M*r^2 - r^3)*ω) + I*cl*r/((2*M*r^2 - r^3)*ω))*H1f(r, ω) - (3*M*r/(2*M*r^2 - r^3) - r^2/(2*M*r^2 - r^3))*Kf(r, ω)

Aqui, para encontrar a solução para $\partial_r \tilde{H}^{(2)}$ faremos dois passos. Primeiro, realizaremos a tranformada de Fourier na eq. (12.370), e em seguida simplificaremos $\partial_r \tilde{K}$ com o uso da eq. (12.372):

In [90]:
eq_Z0_E1 = ((((eq_E1.substitute_function(fun_H)*exp(i*ω*t))/(-i*ω)).simplify().expand())-H1f(r, ω))*(-1)

In [91]:
eq_Z0_E1_1 = mycollect(mycollect(mycollect(mycollect((eq_Z0_E1.rhs().subs(eq_Z0_K)).expand().simplify(), 
            H1f(r, ω)), H0f(r, ω)), Kf(r, ω)), diff(H0f(r, ω), r)) == eq_Z0_E1.lhs()

In [92]:
eq_Z0_E1_1

(-4*I*M^2/((2*M*r^2 - r^3)*ω) + 4*I*M*r/((2*M*r^2 - r^3)*ω) - I*r^2/((2*M*r^2 - r^3)*ω) - 2*I*M/(r^2*ω))*H0f(r, ω) - 1/2*(4*M*cl/((2*M*r^2 - r^3)*ω^2) - 4*M^2*cl/((2*M*r^2 - r^3)*r*ω^2) - cl*r/((2*M*r^2 - r^3)*ω^2) - 2)*H1f(r, ω) + (6*I*M^2/((2*M*r^2 - r^3)*ω) - 5*I*M*r/((2*M*r^2 - r^3)*ω) + I*r^2/((2*M*r^2 - r^3)*ω))*Kf(r, ω) + (2*I*M/(r*ω) - I/ω)*diff(H0f(r, ω), r) == 0

In [93]:
eq_Z0_E1_2 =  eq_Z0_E1_1.solve(diff(H0f(r, ω), r))[0]
#((eq_Z0_E1_1 - (2*i*M/(r*ω) -i/ω)*diff(H0f(r, ω), r))/(-((2*i*M/(r*ω) -i/ω)))).factor()

In [94]:
show(eq_Z0_E1_2)

diff(H0f(r, ω), r) == -1/2*(2*r^3*ω^2*H1f(r, ω) - 2*((4*I*M*r - I*r^2)*H0f(r, ω) + (-3*I*M*r + I*r^2)*Kf(r, ω))*ω + (2*M*cl - cl*r)*H1f(r, ω))/((2*I*M*r^2 - I*r^3)*ω)

In [95]:
eq_Z0_E1_3 = eq_Z0_E1_2.lhs() == mycollect(mycollect(mycollect(eq_Z0_E1_2.rhs().expand(), Kf(r, ω)), H1f(r, ω)), H0f(r, ω))
# eq. 373

In [96]:
show(eq_Z0_E1_3.factor())

diff(H0f(r, ω), r) == -1/2*(-2*I*r^3*ω^2*H1f(r, ω) - 8*M*r*ω*H0f(r, ω) + 2*r^2*ω*H0f(r, ω) + 6*M*r*ω*Kf(r, ω) - 2*r^2*ω*Kf(r, ω) - 2*I*M*cl*H1f(r, ω) + I*cl*r*H1f(r, ω))/((2*M - r)*r^2*ω)

Aqui, para encontrar a solução para $\partial_r \tilde{H}^{(1)}$, faremos a tranformada de Fourier na eq. (12.369) e isolaremos a derivada:

In [97]:
eq_Et_f = ((eq_Et.substitute_function(fun_H)*exp(i*ω*t))/(-i*ω)).expand()

In [98]:
show(eq_Et_f)

Kf(r, ω) == -2*I*M*diff(H1f(r, ω), r)/(r*ω) + I*diff(H1f(r, ω), r)/ω + 2*I*M*H1f(r, ω)/(r^2*ω) - H0f(r, ω)

In [99]:
eq_Et_f_1 = eq_Et_f.solve(diff(H1f(r, ω), r))[0]
# ((eq_Et_f.lhs() == mycollect(eq_Et_f.rhs(), diff(H1f(r, ω), r)))-(-2*I*M/(r*ω) + I/ω)*diff(H1f(r, ω), r) - Kf(r, ω))/(-2*I*M/(r*ω) + I/ω)*(-1)

In [100]:
show(eq_Et_f_1)

diff(H1f(r, ω), r) == ((r^2*H0f(r, ω) + r^2*Kf(r, ω))*ω - 2*I*M*H1f(r, ω))/(-2*I*M*r + I*r^2)

In [101]:
eq_Et_f_2 = eq_Et_f_1.lhs() == mycollect(mycollect(mycollect(eq_Et_f_1.rhs(), H1f(r, ω)), Kf(r, ω)), H0f(r, ω))
# eq. 12.374

In [102]:
show(eq_Et_f_2)

diff(H1f(r, ω), r) == r^2*ω*H0f(r, ω)/(-2*I*M*r + I*r^2) + r^2*ω*Kf(r, ω)/(-2*I*M*r + I*r^2) - 2*I*M*H1f(r, ω)/(-2*I*M*r + I*r^2)

___

Checagem eq. 12.365:

In [103]:
eq_tt_f = ((eq_tt.substitute_function(fun_H)*exp(i*ω*t))).canonicalize_radical() #eq. 12.365 - tranfs. por Fourier

In [104]:
show(eq_tt_f)

diff(Kf(r, ω), r, r) == -1/2*((cl + 2)*H0f(r, ω) + (cl - 2)*Kf(r, ω) - 2*(2*M - r)*diff(H0f(r, ω), r) + 2*(5*M - 3*r)*diff(Kf(r, ω), r))/(2*M*r - r^2)

In [105]:
check1 = (mycollect(mycollect(eq_tt_f.rhs(), diff(Kf(r, ω), r)), diff(H0f(r, ω), r)).subs(eq_Z0_K).subs(eq_Z0_E1_3)).full_simplify()

In [106]:
show(check1)

-1/2*(2*(2*M*r^3 - r^4)*ω^2*H1f(r, ω) - (((I*cl - 2*I)*r^3 - 4*I*M^2*r + 2*(-I*M*cl + 3*I*M)*r^2)*H0f(r, ω) + ((I*cl + 2*I)*r^3 + 18*I*M^2*r + 2*(-I*M*cl - 7*I*M)*r^2)*Kf(r, ω))*ω - (6*M^2*cl - 7*M*cl*r + 2*cl*r^2)*H1f(r, ω))/((4*I*M^2*r^3 - 4*I*M*r^4 + I*r^5)*ω)

In [107]:
check2 = (diff(eq_Z0_K.rhs(), r).subs(eq_Z0_K).subs(eq_Z0_E1_3).subs(eq_Et_f_2)).canonicalize_radical() # usando eq. 372

In [108]:
check2

1/2*(2*(2*M*r^3 - r^4)*ω^2*H1f(r, ω) - (((I*cl - 2*I)*r^3 - 4*I*M^2*r - 2*(I*M*cl - 3*I*M)*r^2)*H0f(r, ω) + ((I*cl + 2*I)*r^3 + 18*I*M^2*r - 2*(I*M*cl + 7*I*M)*r^2)*Kf(r, ω))*ω - (6*M^2*cl - 7*M*cl*r + 2*cl*r^2)*H1f(r, ω))/((-4*I*M^2*r^3 + 4*I*M*r^4 - I*r^5)*ω)

In [109]:
show((check1 - check2).full_simplify())

0

Checagem eq. 12.368:

In [110]:
eq_T0_f = (eq_T0.substitute_function(fun_H)*exp(i*ω*t))/(-i*ω)^2 

In [111]:
eq_T0_f

Kf(r, ω) == -(ω^2*H0f(r, ω)*e^(-I*t*ω) + 2*I*ω*(M/r^2 - 1/r)*H1f(r, ω)*e^(-I*t*ω) + 2*I*ω*(2*M/r - 1)*e^(-I*t*ω)*diff(H1f(r, ω), r) + 2*(2*M/r^2 - 1/r)*e^(-I*t*ω)*diff(H0f(r, ω), r) - (4*M^2/r^2 - 4*M/r + 1)*e^(-I*t*ω)*diff(H0f(r, ω), r, r) + 2*(2*M^2/r^3 - 3*M/r^2 + 1/r)*e^(-I*t*ω)*diff(Kf(r, ω), r) + (4*M^2/r^2 - 4*M/r + 1)*e^(-I*t*ω)*diff(Kf(r, ω), r, r))*e^(I*t*ω)/ω^2

In [112]:
check3 = (eq_T0_f).subs({diff(H0f(r, ω), r, 2) : diff(eq_Z0_E1_3.rhs(), r)}).subs(eq_tt_f).subs(eq_Et_f_2).subs(eq_Z0_E1_3).full_simplify()

In [113]:
check3

Kf(r, ω) == Kf(r, ω)

---

Para encontrar a solução para $\tilde{K}$, usamores a tranformada de Fourier na eq. (12.367), e substituiremos as derivadas $\partial_r \tilde{K}$ e $\partial_r \tilde{H}^{(0)}$ usando as equações (12.371) e (12.372), respectivamente:

In [114]:
eq_L0_f = ((eq_L0.substitute_function(fun_H)*exp(i*ω*t))/(-i*ω)^2).subs(eq_Z0_K).subs(eq_Z0_E1_3).full_simplify()

In [115]:
eq_L0_f

Kf(r, ω) == 1/2*(2*(-2*I*M*r^3 + I*r^4)*ω^2*H1f(r, ω) - (((cl - 2)*r^3 - 12*M^2*r - 2*(M*cl - 5*M)*r^2)*H0f(r, ω) - ((cl - 2)*r^3 - 6*M^2*r - 2*(M*cl - 3*M)*r^2)*Kf(r, ω))*ω + (2*I*M^2*cl - I*M*cl*r)*H1f(r, ω))/(r^5*ω^3)

In [116]:
eq_Z_aux = (((eq_L0_f.rhs() - Kf(r, ω) == 0).canonicalize_radical())*(2*r^5*ω^3)).expand()

In [117]:
eq_Z_aux

-2*r^5*ω^3*Kf(r, ω) - 4*I*M*r^3*ω^2*H1f(r, ω) + 2*I*r^4*ω^2*H1f(r, ω) + 2*M*cl*r^2*ω*H0f(r, ω) - cl*r^3*ω*H0f(r, ω) - 2*M*cl*r^2*ω*Kf(r, ω) + cl*r^3*ω*Kf(r, ω) + 12*M^2*r*ω*H0f(r, ω) - 10*M*r^2*ω*H0f(r, ω) + 2*r^3*ω*H0f(r, ω) - 6*M^2*r*ω*Kf(r, ω) + 6*M*r^2*ω*Kf(r, ω) - 2*r^3*ω*Kf(r, ω) + 2*I*M^2*cl*H1f(r, ω) - I*M*cl*r*H1f(r, ω) == 0

Ao reorganizar a expressão acima, temos a eq. (12.375):

In [118]:
eq_Z = mycollect(mycollect(mycollect(eq_Z_aux.lhs(), Kf(r, ω)), H0f(r, ω)), H1f(r, ω)) == eq_Z_aux.rhs() #eq. 12.375

In [119]:
eq_Z

(2*M*cl*r^2*ω - cl*r^3*ω + 12*M^2*r*ω - 10*M*r^2*ω + 2*r^3*ω)*H0f(r, ω) + (-4*I*M*r^3*ω^2 + 2*I*r^4*ω^2 + 2*I*M^2*cl - I*M*cl*r)*H1f(r, ω) - (2*r^5*ω^3 + 2*M*cl*r^2*ω - cl*r^3*ω + 6*M^2*r*ω - 6*M*r^2*ω + 2*r^3*ω)*Kf(r, ω) == 0

Isolado $\tilde{H}^{(0)}_{lm}\left(r, ω\right)$ na equação acima (eq. 12.375):

In [120]:
eq_H0f = eq_Z.solve(H0f(r, ω))[0]
#eq_H0f = H0f(r, ω) == mycollect(mycollect((eq_Z.lhs() - (2*M*cl*r^2*ω - cl*r^3*ω + 12*M^2*r*ω - 10*M*r^2*ω + 2*r^3*ω)*H0f(r, ω))/(2*M*cl*r^2*ω - cl*r^3*ω + 12*M^2*r*ω - 10*M*r^2*ω + 2*r^3*ω), H1f(r, ω)), Kf(r, ω))

In [121]:
eq_H0f.factor()

H0f(r, ω) == (2*r^5*ω^3*Kf(r, ω) + 4*I*M*r^3*ω^2*H1f(r, ω) - 2*I*r^4*ω^2*H1f(r, ω) + 2*M*cl*r^2*ω*Kf(r, ω) - cl*r^3*ω*Kf(r, ω) + 6*M^2*r*ω*Kf(r, ω) - 6*M*r^2*ω*Kf(r, ω) + 2*r^3*ω*Kf(r, ω) - 2*I*M^2*cl*H1f(r, ω) + I*M*cl*r*H1f(r, ω))/((cl*r + 6*M - 2*r)*(2*M - r)*r*ω)

Substituindo a expressão acima na eq. (12.372):

In [122]:
eq_Z1_aux = eq_Z0_K.rhs().subs(eq_H0f)

In [123]:
eq_Z1_aux

-1/2*(-2*I*M*cl/((2*M*r^2 - r^3)*ω) + I*cl*r/((2*M*r^2 - r^3)*ω))*H1f(r, ω) - (3*M*r/(2*M*r^2 - r^3) - r^2/(2*M*r^2 - r^3))*Kf(r, ω) - (2*r^5*ω^3*Kf(r, ω) - 2*(-2*I*M*r^3 + I*r^4)*ω^2*H1f(r, ω) - ((cl - 2)*r^3 - 6*M^2*r - 2*(M*cl - 3*M)*r^2)*ω*Kf(r, ω) + (-2*I*M^2*cl + I*M*cl*r)*H1f(r, ω))*(2*M*r/(2*M*r^2 - r^3) - r^2/(2*M*r^2 - r^3))/(((cl - 2)*r^3 - 12*M^2*r - 2*(M*cl - 5*M)*r^2)*ω)

Assim, teremos a eq. (12.377):

In [124]:
eq_Z1 = (diff(Kf(r, ω), r) == mycollect(mycollect(eq_Z1_aux, Kf(r, ω)), H1f(r, ω)))
#eq. 12.377

In [125]:
eq_Z1

diff(Kf(r, ω), r) == -1/2*(-16*I*M^2*r^4*ω/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(2*M*r^2 - r^3)) + 16*I*M*r^5*ω/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(2*M*r^2 - r^3)) - 4*I*r^6*ω/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(2*M*r^2 - r^3)) + 8*I*M^3*cl*r/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(2*M*r^2 - r^3)*ω) - 8*I*M^2*cl*r^2/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(2*M*r^2 - r^3)*ω) + 2*I*M*cl*r^3/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(2*M*r^2 - r^3)*ω) - 2*I*M*cl/((2*M*r^2 - r^3)*ω) + I*cl*r/((2*M*r^2 - r^3)*ω))*H1f(r, ω) + (4*M*r^6*ω^2/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(2*M*r^2 - r^3)) - 2*r^7*ω^2/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(2*M*r^2 - r^3)) + 4*M^2*cl*r^3/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(2*M*r^2 - r^3)) - 4*M*cl*r^4/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(2*M*r^2 - r^3)) + cl*r^5/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^

Substituindo a expressão acima nas eq. (12.374):

In [126]:
eq_Z2_aux = eq_Et_f_2.rhs().subs(eq_H0f)

In [127]:
eq_Z2_aux

r^2*ω*Kf(r, ω)/(-2*I*M*r + I*r^2) - (2*r^5*ω^3*Kf(r, ω) - 2*(-2*I*M*r^3 + I*r^4)*ω^2*H1f(r, ω) - ((cl - 2)*r^3 - 6*M^2*r - 2*(M*cl - 3*M)*r^2)*ω*Kf(r, ω) + (-2*I*M^2*cl + I*M*cl*r)*H1f(r, ω))*r^2/(((cl - 2)*r^3 - 12*M^2*r - 2*(M*cl - 5*M)*r^2)*(-2*I*M*r + I*r^2)) - 2*I*M*H1f(r, ω)/(-2*I*M*r + I*r^2)

E a equação (12.378):

In [128]:
eq_Z2 = (diff(H1f(r, ω), r) == mycollect(mycollect(eq_Z2_aux, Kf(r, ω)), H1f(r, ω))) #eq. 12.378

In [129]:
eq_Z2

diff(H1f(r, ω), r) == (4*I*M*r^5*ω^2/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(-2*I*M*r + I*r^2)) - 2*I*r^6*ω^2/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(-2*I*M*r + I*r^2)) - 2*I*M^2*cl*r^2/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(-2*I*M*r + I*r^2)) + I*M*cl*r^3/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(-2*I*M*r + I*r^2)) - 2*I*M/(-2*I*M*r + I*r^2))*H1f(r, ω) + (2*r^7*ω^3/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(-2*I*M*r + I*r^2)) + 2*M*cl*r^4*ω/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(-2*I*M*r + I*r^2)) - cl*r^5*ω/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(-2*I*M*r + I*r^2)) + 6*M^2*r^3*ω/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(-2*I*M*r + I*r^2)) - 6*M*r^4*ω/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(-2*I*M*r + I*r^2)) + 2*r^5*ω/((2*M*cl*r^2 - cl*r^3 + 12*M^2*r - 10*M*r^2 + 2*r^3)*(-2*I*M*r + I*r^2)) + r^2*ω/(-2*I*M*r + I*r^2))*Kf(r, ω)

Aqui, vamos encontrar uma solução apenas em função de $\tilde{K}_{lm}$:

Definindo uma nova variável $R(r)$, juntamente com um fator $\omega$, no lugar de $\tilde{H}^{(1)}$:

In [130]:
Rf(r) = function('Rf', latex_name=r'\tilde{R}_{lm}')(r)

In [131]:
Rdef = {H1f(r) : ω*Rf(r)}

In [132]:
Rdef[diff(list(Rdef.keys())[0], r)] = diff(list(Rdef.values())[0], r)

In [133]:
Rdef

{H1f(r, ω): ω*Rf(r), diff(H1f(r, ω), r): ω*diff(Rf(r), r)}

Definindo uma expressão para $\partial_r \tilde{K}(r, \omega)$, com as novas regras de substiuição:

In [134]:
Zeq1_K = (diff(Kf(r), r) - eq_Z1).substitute_function(Rdef).factor().rhs()

In [135]:
Zeq1_K

-1/2*(4*r^5*ω^2*Kf(r, ω) + 8*I*M*r^3*ω^2*Rf(r) - 4*I*r^4*ω^2*Rf(r) - 4*M*cl*r^3*diff(Kf(r, ω), r) + 2*cl*r^4*diff(Kf(r, ω), r) - 2*M*cl*r^2*Kf(r, ω) + 2*I*M*cl^2*r*Rf(r) - I*cl^2*r^2*Rf(r) - 24*M^2*r^2*diff(Kf(r, ω), r) + 20*M*r^3*diff(Kf(r, ω), r) - 4*r^4*diff(Kf(r, ω), r) - 24*M^2*r*Kf(r, ω) + 12*M*r^2*Kf(r, ω) + 8*I*M^2*cl*Rf(r) - 8*I*M*cl*r*Rf(r) + 2*I*cl*r^2*Rf(r))/((cl*r + 6*M - 2*r)*(2*M - r)*r^2)

Assim como uma expressão para $\partial_r \tilde{H}^{(1)}(r, \omega)$:

In [136]:
Zeq1_R = (diff(H1f(r), r) - eq_Z2).substitute_function(Rdef).factor().rhs()

In [137]:
Zeq1_R

-I*(2*r^6*ω^2*Kf(r, ω) + 4*I*M*r^4*ω^2*Rf(r) - 2*I*r^5*ω^2*Rf(r) + 4*M*cl*r^3*Kf(r, ω) - 2*cl*r^4*Kf(r, ω) + 4*I*M^2*cl*r^2*diff(Rf(r), r) - 4*I*M*cl*r^3*diff(Rf(r), r) + I*cl*r^4*diff(Rf(r), r) + 18*M^2*r^2*Kf(r, ω) - 16*M*r^3*Kf(r, ω) + 4*r^4*Kf(r, ω) - 6*I*M^2*cl*r*Rf(r) + 3*I*M*cl*r^2*Rf(r) + 24*I*M^3*r*diff(Rf(r), r) - 32*I*M^2*r^2*diff(Rf(r), r) + 14*I*M*r^3*diff(Rf(r), r) - 2*I*r^4*diff(Rf(r), r) - 24*I*M^3*Rf(r) + 20*I*M^2*r*Rf(r) - 4*I*M*r^2*Rf(r))*ω/((cl*r + 6*M - 2*r)*(2*M - r)^2*r)

Aqui, definiremos quatro novas funções:

In [138]:
f1 = function('f1', latex_name=r'f_1')(r)
f2 = function('f2', latex_name=r'f_2')(r)
f3 = function('f3', latex_name=r'f_3')(r)
f4 = function('f4', latex_name=r'f_4')(r)

De modo que as antigas variáveis $\tilde{K}(r, \omega)$ e $\tilde{R}(r, \omega)$ são substiuídas pelas expressões:

In [139]:
eq_Kn = {Kf : f1*Kf + f2*Rf}
eq_Rn = {Rf : f3*Kf + f4*Rf}

In [140]:
eq_Kn[diff(list(eq_Kn.keys())[0], r)] = diff(list(eq_Kn.values())[0], r)

In [141]:
eq_Kn

{(r, ω) |--> Kf(r, ω): (r, ω) |--> Kf(r, ω)*f1(r) + Rf(r)*f2(r),
 (r, ω) |--> diff(Kf(r, ω), r): (r, ω) |--> f1(r)*diff(Kf(r, ω), r) + f2(r)*diff(Rf(r), r) + Kf(r, ω)*diff(f1(r), r) + Rf(r)*diff(f2(r), r)}

In [142]:
eq_Rn[diff(list(eq_Rn.keys())[0], r)] = diff(list(eq_Rn.values())[0], r)

In [143]:
eq_Rn

{r |--> Rf(r): (r, ω) |--> Kf(r, ω)*f3(r) + Rf(r)*f4(r),
 r |--> diff(Rf(r), r): (r, ω) |--> f3(r)*diff(Kf(r, ω), r) + f4(r)*diff(Rf(r), r) + Kf(r, ω)*diff(f3(r), r) + Rf(r)*diff(f4(r), r)}

In [144]:
Zeq1replaced_K = Zeq1_K.subs(eq_Kn, eq_Rn).factor()

In [145]:
Zeq1replaced_R = Zeq1_R.subs(eq_Kn, eq_Rn).factor()

Aqui, definiremos a coordenada de tartaruga, $r_\star$:

In [146]:
diff_Tor = {diff(ftortoise, r) : 1/sqrt(f/B)}

In [147]:
diff_Tor[diff(list(diff_Tor.keys())[0], r)] = diff(list(diff_Tor.values())[0], r) 

In [148]:
show(diff_Tor)

{diff(ftortoise(r), r): 1/sqrt((2*M/r - 1)^2),
 diff(ftortoise(r), r, r): 2*M*(2*M/r - 1)/(r^2*((2*M/r - 1)^2)^(3/2))}

In [149]:
Kt(tortoise, ω) = function('Kt', latex_name=r'\tilde{K}_{lm}')(tortoise, ω)
Rt(tortoise, ω) = function('Rt', latex_name=r'\tilde{R}_{lm}')(tortoise, ω)
Kt, Rt

((tortoise, ω) |--> Kt(tortoise, ω), (tortoise, ω) |--> Rt(tortoise, ω))

In [150]:
fun_diff = {Kf(r, ω) : Kt(tortoise, ω), diff(Kf(r, ω), r) : diff(Kt(tortoise, ω), tortoise)*diff(ftortoise, r),
           Rf(r, ω) : Rt(tortoise, ω), diff(Rf(r, ω), r) : diff(Rt(tortoise, ω), tortoise)*diff(ftortoise, r)}
show(fun_diff)

{Kf(r, ω): Kt(tortoise, ω),
 diff(Kf(r, ω), r): diff(Kt(tortoise, ω), tortoise)*diff(ftortoise(r), r),
 Rf(r): Rt(tortoise, ω),
 diff(Rf(r), r): diff(Rt(tortoise, ω), tortoise)*diff(ftortoise(r), r)}

Substituindo ambas as expressões acima nas equações para $\tilde{K}$ e para $\tilde{R}$, bem como a regra de substituição para as derivadas de $r_{\star}$:

In [151]:
Zeq1tor_Kn = Zeq1replaced_K.subs(fun_diff)

In [152]:
Zeq1tor_Kn = Zeq1tor_Kn.subs(diff_Tor)

In [153]:
#Zeq1tor_Kn

In [154]:
Zeq1tor_Rn = Zeq1replaced_R.subs(fun_diff)

In [155]:
Zeq1tor_Rn = Zeq1tor_Rn.subs(diff_Tor)

In [156]:
#Zeq1tor_Rn

Aqui, temos de resolver um sistema de duas equações, em termos das funções $\partial_\star\tilde{K}_{lm}$ e $\partial_{\star}\tilde{R}_{lm}$, como se segue:

In [157]:
#Kn, Rn
dic_aux2 = dict(zip([diff(Kt(tortoise, ω), tortoise), diff(Rt(tortoise, ω), tortoise)], var('aux2', n=3)))

In [158]:
list(dic_aux2.values())

[aux20, aux21]

In [159]:
sist2 = solve([Zeq1tor_Kn.subs(dic_aux2)==0, Zeq1tor_Rn.subs(dic_aux2)==0], list(dic_aux2.values()))

a partir daqui, assumiremos que $f_2(r) = 1$ por simplicidade:

In [160]:
Zeqsol = [diff(Kt(tortoise), tortoise) == sist2[0][0].rhs().subs({f2 : 1, diff(f2, r) : 0}), diff(Rt(tortoise), tortoise) == sist2[0][1].rhs().subs({f2 : 1, diff(f2, r) : 0})]

In [161]:
Zeqsol_subs = {diff(Kt(tortoise), tortoise) : sist2[0][0].rhs().subs({f2 : 1, diff(f2, r) : 0}), diff(Rt(tortoise), tortoise) : sist2[0][1].rhs().subs({f2 : 1, diff(f2, r) : 0})}

Aqui, formularemos dois requisitos sobre a transformação:

In [162]:
requisito1 = (diff(Kt, tortoise) - Rt).subs(Zeqsol_subs)

In [163]:
requisito2 = (diff(Rt, tortoise).subs(Zeqsol_subs)).coefficient(Rt)

As requisitos acima levam à equações algébricas e diferenciais para $f_1(r)$, $f_3(r)$ e $f_4(r)$. Assim, basta pegar as equações algébricas e depois verificar as diferenciais. Começaremos pela resolução da expressão para $f_4(r)$, nos restringindo aos coeficientes de $\tilde{R}(r_{\star})$ e $\omega^2$, nessa ordem:

In [171]:
f4sol = ((requisito1.coefficient(Rt)).coefficient(ω^2) == 0).solve(f4, solution_dict=True)[0]

In [172]:
f4sol[diff(list(f4sol.keys())[0], r)] = diff(list(f4sol.values())[0], r)

In [173]:
show(f4sol)

{f4(r): r^2/(-2*I*M + I*r),
 diff(f4(r), r): 2*r/(-2*I*M + I*r) - I*r^2/(-2*I*M + I*r)^2}

Para a solução de $f_3(r)$, substituiremos o resultado acima no `requisito1` e então pegaremos os coeficientes de $\tilde{R}(r_{\star})$:

In [171]:
f3sol = ((requisito1).subs(f4sol).coefficient(Rt) == 0).solve(f3, solution_dict=True)[0]

In [172]:
f3sol[diff(list(f3sol.keys())[0], r)] = diff(list(f3sol.values())[0], r)

In [173]:
f3sol

{f3(r): -((cl - 2)*r^2*sqrt((2*M/r - 1)^2) + 6*M*r*sqrt((2*M/r - 1)^2) + 2*(2*M*r^2 - r^3)*f1(r))/(8*I*M^2 - 8*I*M*r + 2*I*r^2),
 diff(f3(r), r): -2*((cl - 2)*r*sqrt((2*M/r - 1)^2) - M*(cl - 2)*(2*M/r - 1)/sqrt((2*M/r - 1)^2) + (4*M*r - 3*r^2)*f1(r) + (2*M*r^2 - r^3)*diff(f1(r), r) + 3*M*sqrt((2*M/r - 1)^2) - 6*M^2*(2*M/r - 1)/(r*sqrt((2*M/r - 1)^2)))/(8*I*M^2 - 8*I*M*r + 2*I*r^2) - 4*((cl - 2)*r^2*sqrt((2*M/r - 1)^2) + 6*M*r*sqrt((2*M/r - 1)^2) + 2*(2*M*r^2 - r^3)*f1(r))*(2*I*M - I*r)/(8*I*M^2 - 8*I*M*r + 2*I*r^2)^2}

O mesmo procedimento será realizado para a função $f_1(r)$, mas usando o `requesito2`, como se segue:

In [174]:
f1sol = (requisito2.subs(f4sol).subs(f3sol) == 0).solve(f1, solution_dict=True)[0]

In [175]:
f1sol[diff(list(f1sol.keys())[0], r)] = diff(list(f1sol.values())[0], r)

In [176]:
f1sol

{f1(r): 1/2*((cl^2 - 2*cl)*r^2*sqrt((2*M/r - 1)^2) + 24*M^2*sqrt((2*M/r - 1)^2) + 6*(M*cl - 2*M)*r*sqrt((2*M/r - 1)^2))/((cl - 2)*r^3 - 12*M^2*r - 2*(M*cl - 5*M)*r^2),
 diff(f1(r), r): -1/2*((cl^2 - 2*cl)*r^2*sqrt((2*M/r - 1)^2) + 24*M^2*sqrt((2*M/r - 1)^2) + 6*(M*cl - 2*M)*r*sqrt((2*M/r - 1)^2))*(3*(cl - 2)*r^2 - 12*M^2 - 4*(M*cl - 5*M)*r)/((cl - 2)*r^3 - 12*M^2*r - 2*(M*cl - 5*M)*r^2)^2 + ((cl^2 - 2*cl)*r*sqrt((2*M/r - 1)^2) - (cl^2 - 2*cl)*M*(2*M/r - 1)/sqrt((2*M/r - 1)^2) + 3*(M*cl - 2*M)*sqrt((2*M/r - 1)^2) - 24*M^3*(2*M/r - 1)/(r^2*sqrt((2*M/r - 1)^2)) - 6*(M*cl - 2*M)*M*(2*M/r - 1)/(r*sqrt((2*M/r - 1)^2)))/((cl - 2)*r^3 - 12*M^2*r - 2*(M*cl - 5*M)*r^2)}

Aqui, inseriremos a solução das três funções nas expressões para $\tilde{K}(r_{\star})$ e $\tilde{R}(r_{\star})$, dentro da lista `Zeqsol`:

In [177]:
Zeqsol1_Kn = (Zeqsol[0].subs(f4sol).subs(f3sol).subs(f1sol)).simplify()

In [178]:
Zeqsol1_Rn = (Zeqsol[1].subs(f4sol).subs(f3sol).subs(f1sol)).simplify()

In [179]:
#print(Zeqsol1_Kn)

Definindo uma nova variável $\lambda$ e definindo-a como $l(l+1) = 2(\lambda+1)$ e aplicando as substituição `backg`, tem-se:

In [180]:
var('λ')
assume(r>0)

In [181]:
Zeqs = [Zeqsol1_Kn.subs({cl : 2*(λ+1)}).factor(), Zeqsol1_Rn.subs({cl : 2*(λ+1)}).factor()]

In [182]:
Zeqs

[diff(Kt(tortoise, ω), tortoise) == Rt(tortoise, ω),
 diff(Rt(tortoise, ω), tortoise) == -(r^6*λ^2*ω^2 + 6*M*r^5*λ*ω^2 + 9*M^2*r^4*ω^2 + 4*M*r^3*λ^3 - 2*r^4*λ^3 + 12*M^2*r^2*λ^2 - 2*M*r^3*λ^2 - 2*r^4*λ^2 + 36*M^3*r*λ - 18*M^2*r^2*λ + 36*M^4 - 18*M^3*r)*Kt(tortoise, ω)/((r*λ + 3*M)^2*r^4)]

In [183]:
Zeqsol1_Rn.subs({cl : 2*(λ+1)}).factor()

diff(Rt(tortoise, ω), tortoise) == -(r^6*λ^2*ω^2 + 6*M*r^5*λ*ω^2 + 9*M^2*r^4*ω^2 + 4*M*r^3*λ^3 - 2*r^4*λ^3 + 12*M^2*r^2*λ^2 - 2*M*r^3*λ^2 - 2*r^4*λ^2 + 36*M^3*r*λ - 18*M^2*r^2*λ + 36*M^4 - 18*M^3*r)*Kt(tortoise, ω)/((r*λ + 3*M)^2*r^4)

In [184]:
Zeqs_subs = {diff(Rt(tortoise), tortoise) : Zeqs[1].rhs()}

In [185]:
Zeqs_subs

{diff(Rt(tortoise, ω), tortoise): -(r^6*λ^2*ω^2 + 6*M*r^5*λ*ω^2 + 9*M^2*r^4*ω^2 + 4*M*r^3*λ^3 - 2*r^4*λ^3 + 12*M^2*r^2*λ^2 - 2*M*r^3*λ^2 - 2*r^4*λ^2 + 36*M^3*r*λ - 18*M^2*r^2*λ + 36*M^4 - 18*M^3*r)*Kt(tortoise, ω)/((r*λ + 3*M)^2*r^4)}

Aqui, transformaremos duas equações de primeira ordem em uma de segunda ordem:

In [186]:
expr = diff(Kt(tortoise), tortoise, 2) - (diff(Rt(tortoise), tortoise).subs(Zeqs_subs)) == 0

In [187]:
expr

(r^6*λ^2*ω^2 + 6*M*r^5*λ*ω^2 + 9*M^2*r^4*ω^2 + 4*M*r^3*λ^3 - 2*r^4*λ^3 + 12*M^2*r^2*λ^2 - 2*M*r^3*λ^2 - 2*r^4*λ^2 + 36*M^3*r*λ - 18*M^2*r^2*λ + 36*M^4 - 18*M^3*r)*Kt(tortoise, ω)/((r*λ + 3*M)^2*r^4) + diff(Kt(tortoise, ω), tortoise, tortoise) == 0

Por fim, aplicaremos a função `canonicalize_de()`para encontrar uma equação de onda do tipo Schödinger:

In [188]:
ZE = canonicalize_de(expr, Kt(tortoise), tortoise)

canonicalize_de for Kt(tortoise, ω): passed!


In [189]:
ZE

(r^6*λ^2*ω^2 + 6*M*r^5*λ*ω^2 + 9*M^2*r^4*ω^2 + 4*M*r^3*λ^3 - 2*r^4*λ^3 + 12*M^2*r^2*λ^2 - 2*M*r^3*λ^2 - 2*r^4*λ^2 + 36*M^3*r*λ - 18*M^2*r^2*λ + 36*M^4 - 18*M^3*r)*Kt(tortoise, ω)/((r*λ + 3*M)^2*r^4) + diff(Kt(tortoise, ω), tortoise, tortoise) == 0

Comparando a expressão acima com 

$$\left(\omega^2 - V\right)\hat{K} + \hat{K}''=0$$ 

temos para a expressão do potencial

In [190]:
V_ZE(r) = (ω^2 - get_de_coefficients(ZE, Kt(tortoise), tortoise)[0].expand()).factor()
show(LatexExpr(r'V^{ZE}_\ell(r) = '),V_ZE(r))

V^{ZE}_\ell(r) =  -2*(r^3*λ^3 + 3*M*r^2*λ^2 + r^3*λ^2 + 9*M^2*r*λ + 9*M^3)*(2*M - r)/((r*λ + 3*M)^2*r^4)

In [191]:
stage_time()

Total time elapsed until the end of stage 5: 00:05:11
